# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [1]:
!pip install langchain_community langchain_experimental langchain_openai

In [2]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [6]:
!pip install faiss-cpu
!pip install google-search-results
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
    --------------------------------------- 0.3/18.2 MB ? eta -:--:--
   - -------------------------------------- 0.8/18.2 MB 2.2 MB/s eta 0:00:08
   -- ------------------------------------- 1.0/18.2 MB 2.4 MB/s eta 0:00:08
   -- ------------------------------------- 1.3/18.2 MB 1.8 MB/s eta 0:00:10
   --- ------------------------------------ 1.6/18.2 MB 1.5 MB/s eta 0:00:12
   ---- ----------------------------------- 1.8/18.2 MB 1.5 MB/s eta 0:00:11
   ---- ----------------------------------- 2.1/18.2 MB 1.4 MB/s eta 0:00:12
   ----- ---------------------------------- 2.4/18.2 MB 1.4 MB/s eta 0:00:12
   ----- ---------------------------------- 2.4/18.2 MB 1.4 MB/s eta 0:00:12
   ----- ---------------------------------- 2.6/18.2 MB 1.2 MB/s eta 0:00:14
   ----- ---------------------------------- 2.6/18.2 MB 1.2 MB/s eta 0:00:14
   ------ --------------------------------- 2.9/18.2 MB 1.1 MB/s eta 0:00:14
   ------ ---

  DEPRECATION: Building 'google-search-results' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'google-search-results'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [7]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv("env")

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [19]:
from langchain_openai import OpenAIEmbeddings
# Define your embedding model
embeddings_model = OpenAIEmbeddings()

# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [1]:
from serpapi import GoogleSearch
print("Import OK:", GoogleSearch)







Import OK: <class 'serpapi.google_search.GoogleSearch'>


Import OK: <class 'serpapi.google_search.GoogleSearch'>


## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [7]:
!pip install -U langchain langchain-community langchain-openai serpapi python-dotenv

import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())  # loads OPENAI_API_KEY and SERPAPI_API_KEY

# ---- LangChain imports
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import ChatOpenAI  # use chat models

# ---- Check keys (optional but helpful)
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY not set.")
if not os.getenv("SERPAPI_API_KEY"):
    raise RuntimeError("SERPAPI_API_KEY not set.")

OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

# ---- Tool 1: TODO (LLMChain)
todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. "
    "Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=ChatOpenAI(temperature=0, model=OPENAI_MODEL), prompt=todo_prompt)

# ---- Tool 2: Search (SerpAPI)
search = SerpAPIWrapper(serpapi_api_key=os.getenv("SERPAPI_API_KEY"))

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful for answering questions about current events or facts on the web.",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="Useful for creating todo lists. Input: an objective. Output: a clear todo list.",
    ),
]

# ---- Agent prompt
prefix = (
    "You are an AI who performs one task based on the following objective: {objective}. "
    "Take into account these previously completed tasks: {context}."
)
suffix = "Question: {task}\n{agent_scratchpad}"

agent_prompt = ZeroShotAgent.create_prompt(
    tools=tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

# ---- Build agent + executor
llm_for_agent = ChatOpenAI(temperature=0, model=OPENAI_MODEL)
llm_chain = LLMChain(llm=llm_for_agent, prompt=agent_prompt)

agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=[t.name for t in tools])
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

# ---- Run
inputs = {
    "objective": "Plan a productive Saturday at home on a rainy day.",
    "task": "Create a concise, prioritized to-do list with rough times.",
    "context": "I already bought groceries and have 5 hours free.",
}
result = agent_executor.invoke(inputs)
print("\n=== RESULT ===\n")
print(result.get("output", result))


C:\Users\katyd\AppData\Local\Temp\ipykernel_20056\1439212814.py:27: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  todo_chain = LLMChain(llm=ChatOpenAI(temperature=0, model=OPENAI_MODEL), prompt=todo_prompt)
C:\Users\katyd\AppData\Local\Temp\ipykernel_20056\1439212814.py:63: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.



> Entering new AgentExecutor chain...
Thought: I need to create a productive Saturday plan for a rainy day at home, considering I have 5 hours free and groceries are already bought. I should include a mix of tasks that are enjoyable and productive. 

Action: TODO
Action Input: Create a to-do list for a productive Saturday at home on a rainy day, including time estimates for each task.
Observation: Here's a productive to-do list for a rainy Saturday at home, complete with time estimates for each task:

### Productive Saturday To-Do List

**Morning:**

1. **Wake Up & Morning Routine (8:00 AM - 9:00 AM)**  
   - Brush teeth, shower, and get dressed (30 minutes)  
   - Prepare and enjoy breakfast (30 minutes)  

2. **Plan the Day (9:00 AM - 9:30 AM)**  
   - Review tasks and set priorities for the day (30 minutes)  

3. **Declutter Living Space (9:30 AM - 10:30 AM)**  
   - Tidy up common areas (30 minutes)  
   - Organize one specific area (e.g., a drawer or shelf) (30 minutes)  

**Mid

In [14]:
from langchain_openai import ChatOpenAI  # not OpenAI

llm = ChatOpenAI(temperature=0, model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"))
llm_chain = LLMChain(llm=llm, prompt=agent_prompt)  # <- use the prompt you created above

agent = ZeroShotAgent(
    llm_chain=llm_chain,
    allowed_tools=[t.name for t in tools],
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

# RUN IT
inputs = {
    "objective": "Plan a productive Saturday at home on a rainy day.",
    "task": "Create a concise, prioritized to-do list with rough times.",
    "context": "I already bought groceries and have 5 hours free.",
}
result = agent_executor.invoke(inputs)
print(result.get("output", result))




> Entering new AgentExecutor chain...
Thought: I need to create a productive Saturday plan that fits within the 5 hours available and takes advantage of being at home on a rainy day. I should consider activities that are enjoyable and fulfilling, such as cooking, reading, or organizing. 

Action: TODO
Action Input: Create a to-do list for a productive Saturday at home on a rainy day, including time estimates for each task.

Observation: Here's a productive Saturday to-do list for a rainy day at home, complete with time estimates for each task:

### Productive Saturday To-Do List

**Morning:**

1. **Wake Up & Morning Routine (8:00 AM - 9:00 AM)**  
   - Brush teeth, shower, and get dressed (30 minutes)  
   - Prepare and enjoy breakfast (30 minutes)  

2. **Plan the Day (9:00 AM - 9:30 AM)**  
   - Review tasks and set priorities for the day (30 minutes)  

3. **Declutter Living Space (9:30 AM - 10:30 AM)**  
   - Tidy up common areas (30 minutes)  
   - Organize one specific area (e.

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [14]:
OBJECTIVE = "Write a weather report for SF today"

In [15]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [16]:
baby_agi({"objective": OBJECTIVE})

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should create a todo list based on the given objective
Action: TODO
Action Input: Create a todo list for SF weather report
Observation: 

1. Research the current weather patterns in San Francisco.
2. Determine the sources for obtaining accurate weather information for the city.
3. Create a list of key weather elements to include in the report, such as temperature, precipitation, wind speed, and humidity.
4. Decide on the format of the report, whether it will be a written document, a presentation, or a combination of both.
5. Set a timeline for when the report needs to be completed and shared.
6. Gather data from reliable sources, such as the National Weather Service or local meteorologists.
7. Organize the data into a clear and easy-to-understand format.
8. Include any relevant graphics or visuals to enhance the report.
9. Proofread and edit the report 

{'objective': 'Write a weather report for SF today'}